<a href="https://colab.research.google.com/github/Mini1521/Sentiment-Analysis/blob/main/Using_Microsoft_and_Tesla_Dataset_from_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Data- preprocessing

In [6]:
import pandas as pd

# Load the two CSV files
file1 = "/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Company.csv"  # Replace with your actual file name
file2 = "/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/CompanyValues.csv"  # Replace with your actual file name

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Merge the data on a common column (e.g., 'day_date')
merged_df = pd.merge(df1, df2, on="ticker_symbol", how="left")  # Use "outer", "left", or "right" if needed

# Save the merged dataset to a new CSV file
#merged_df.to_csv("merged_data.csv", index=False)

print("Merged CSV saved as 'merged_data.csv'")


Merged CSV saved as 'merged_data.csv'


In [7]:
import pandas as pd

# Load the dataset (update the filename if needed)
file_path = "/content/drive/MyDrive/Colab Notebooks/Datasets/merged_data.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Convert 'day_date' to datetime format
df['day_date'] = pd.to_datetime(df['day_date'], errors='coerce')

# Filter data for the year 2020
df_2020 = df[df['day_date'].dt.year == 2020]

# Separate data for Microsoft (MSFT) and Tesla (TSLA)
msft_data = df_2020[df_2020['ticker_symbol'] == 'MSFT']
tsla_data = df_2020[df_2020['ticker_symbol'] == 'TSLA']

# Drop rows with missing values (optional)
msft_data = msft_data.dropna()
tsla_data = tsla_data.dropna()

# Save each dataset separately
#msft_data.to_csv("msft_2020.csv", index=False)
#tsla_data.to_csv("tsla_2020.csv", index=False)

print("Data preprocessing complete.")
print("Saved: 'msft_2020.csv' and 'tsla_2020.csv'")


Data preprocessing complete.
Saved: 'msft_2020.csv' and 'tsla_2020.csv'


# Social Media - preprocessing

In [8]:
import numpy as np
import pandas as pd

# Load datasets
company_tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Company_Tweet.csv")
tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Tweet.csv")

In [9]:
# Merge tweets with company information
tweets = tweets.merge(company_tweets, how='left', on='tweet_id')

In [10]:
# Convert 'post_date' to datetime format
tweets['date'] = pd.to_datetime(tweets['post_date'], unit='s').dt.date
tweets['date'] = pd.to_datetime(tweets['date'], errors='coerce')
tweets['time'] = pd.to_datetime(tweets['post_date'], unit='s').dt.time

In [11]:
# Filter for Microsoft (MSFT) and Tesla (TSLA)
tweets_filtered = tweets[tweets['ticker_symbol'].isin(['MSFT', 'TSLA'])]

# Keep only tweets from 2020
tweets_filtered = tweets_filtered[tweets_filtered['date'].dt.year == 2020]

In [12]:
# Split into separate datasets
msft_tweets = tweets_filtered[tweets_filtered['ticker_symbol'] == 'MSFT']
tsla_tweets = tweets_filtered[tweets_filtered['ticker_symbol'] == 'TSLA']

In [8]:
# Save to CSV files
msft_tweets.to_csv("msft_tweets_2020.csv", index=False)
tsla_tweets.to_csv("tsla_tweets_2020.csv", index=False)

**cleaning of microsoft tweets**

In [13]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
# Load Twitter dataset
tweets_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/msft_tweets_2020.csv")

In [15]:
# Check dataset info
print(tweets_df.head())
#print(tweets_df.info())
print(tweets_df.columns)

Empty DataFrame
Columns: [tweet_id, writer, post_date, body, comment_num, retweet_num, like_num, ticker_symbol, date, time]
Index: []
Index(['tweet_id', 'writer', 'post_date', 'body', 'comment_num', 'retweet_num',
       'like_num', 'ticker_symbol', 'date', 'time'],
      dtype='object')


In [16]:
# rename columns
tweets_df.rename(columns={'body': 'Tweet'}, inplace=True)
tweets_df.rename(columns={'post_date': 'day_date'}, inplace=True)

In [17]:
# Droping Duplicates
tweets_df.drop_duplicates(subset=['Tweet'], inplace=True)

,tweet_id,writer,day_date,Tweet,comment_num,retweet_num,like_num,ticker_symbol,date,time


In [18]:
# Handle Missing Values
tweets_df.dropna(subset=['Tweet'], inplace=True)  # Drop rows where 'Tweet' is empty

In [19]:
stop_words = set(stopwords.words('english'))
# Text Preprocessing (Cleaning Tweets)
def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Removing URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Removing mentions and hashtags
    text = re.sub(r'\d+', '', text)  # Removing numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Removing punctuation
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Removing stopwords
    return ' '.join(tokens)

In [20]:
# Apply cleaning function
tweets_df['cleaned_tweet'] = tweets_df['Tweet'].apply(clean_text)

In [17]:
#Saving the cleaned data for further analysis
#tweets_df.to_csv("cleaned_msft_tweets.csv", index=False)

**cleaning tesla tweets**

In [21]:
# Load Twitter dataset
tweets_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/tsla_tweets_2020.csv")

In [22]:
# Check dataset info
print(tweets_df.head())
#print(tweets_df.info())
print(tweets_df.columns)

Empty DataFrame
Columns: [tweet_id, writer, post_date, body, comment_num, retweet_num, like_num, ticker_symbol, date, time]
Index: []
Index(['tweet_id', 'writer', 'post_date', 'body', 'comment_num', 'retweet_num',
       'like_num', 'ticker_symbol', 'date', 'time'],
      dtype='object')


In [23]:
# rename columns
tweets_df.rename(columns={'body': 'Tweet'}, inplace=True)
tweets_df.rename(columns={'post_date': 'day_date'}, inplace=True)

In [24]:
# Droping Duplicates
tweets_df.drop_duplicates(subset=['Tweet'], inplace=True)

,tweet_id,writer,day_date,Tweet,comment_num,retweet_num,like_num,ticker_symbol,date,time


In [25]:
# Handle Missing Values
tweets_df.dropna(subset=['Tweet'], inplace=True)  # Drop rows where 'Tweet' is empty

In [26]:
stop_words = set(stopwords.words('english'))
# Text Preprocessing (Cleaning Tweets)
def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Removing URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Removing mentions and hashtags
    text = re.sub(r'\d+', '', text)  # Removing numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Removing punctuation
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Removing stopwords
    return ' '.join(tokens)

In [27]:
# Apply cleaning function
tweets_df['cleaned_tweet'] = tweets_df['Tweet'].apply(clean_text)

In [25]:
#Saving the cleaned data for further analysis
#tweets_df.to_csv("cleaned_tsla_tweets.csv", index=False)

# Microsoft Articles - Preprocessing

In [28]:
import pandas as pd

# Load the dataset (Update the filename if needed)
article = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/msft_articles.csv")

In [29]:
# Convert 'date' column to datetime format
article['date'] = pd.to_datetime(article['date'], errors='coerce')

# Filter for only the year 2020
year_2020 = article[article['date'].dt.year == 2020]

# Drop rows with missing values
year_2020 = year_2020.dropna()

In [30]:
# Text preprocessing function
def clean_text(text):
    import re
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.strip()  # Remove extra spaces
    return text

In [31]:
# Apply text cleaning to the 'text' column
year_2020['text'] = year_2020['text'].astype(str).apply(clean_text)

# Save the cleaned dataset
#year_2020.to_csv("cleaned_msft_articles.csv", index=False)

# Tesla Articles - Preprocessing

In [32]:
import pandas as pd

# Load the dataset (Update the filename if needed)
article = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/tsla_articles.csv')

In [33]:
# Convert 'date' column to datetime format
article['date'] = pd.to_datetime(article['date'], errors='coerce')

# Filter for only the year 2020
year_2020 = article[article['date'].dt.year == 2020]

# Drop rows with missing values
year_2020 = year_2020.dropna()

In [34]:
# Text preprocessing function
def clean_text(text):
    import re
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.strip()  # Remove extra spaces
    return text

In [35]:
# Apply text cleaning to the 'text' column
year_2020['text'] = year_2020['text'].astype(str).apply(clean_text)

# Save the cleaned dataset
#year_2020.to_csv("cleaned_tsla_articles.csv", index=False)

# Sentiment Analysis- microsoft

In [5]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Load datasets
tweets_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/cleaned_msft_tweets.csv")  # Your processed tweets dataset
news_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/cleaned_msft_articles.csv")  # Your processed news dataset

# Ensure text column is string type
tweets_df['Tweet'] = tweets_df['Tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

In [6]:
# VADER Sentiment Analysis for Tweets

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Function to get sentiment score from VADER
def get_vader_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply VADER sentiment analysis to tweets
tweets_df['sentiment_vader'] = tweets_df['Tweet'].apply(get_vader_sentiment)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
# TextBlob Sentiment Analysis for News

# Function to get sentiment score from TextBlob
def get_textblob_sentiment(text):
    score = TextBlob(text).sentiment.polarity
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply TextBlob sentiment analysis to news articles
news_df['sentiment_textblob'] = news_df['text'].apply(get_textblob_sentiment)

# Save sentiment-labeled datasets
tweets_df.to_csv("sentiment_msft_tweets.csv", index=False)
news_df.to_csv("sentiment_msft_news.csv", index=False)

print("Sentiment analysis complete! Results saved.")

Sentiment analysis complete! Results saved.


# Sentiment Analysis- tesla

In [8]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Load datasets
tweets_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/cleaned_tsla_tweets.csv")  # Your processed tweets dataset
news_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/cleaned_tsla_articles.csv")  # Your processed news dataset

# Ensure text column is string type
tweets_df['Tweet'] = tweets_df['Tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

In [9]:
# VADER Sentiment Analysis for Tweets

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Function to get sentiment score from VADER
def get_vader_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply VADER sentiment analysis to tweets
tweets_df['sentiment_vader'] = tweets_df['Tweet'].apply(get_vader_sentiment)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
# TextBlob Sentiment Analysis for News

# Function to get sentiment score from TextBlob
def get_textblob_sentiment(text):
    score = TextBlob(text).sentiment.polarity
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply TextBlob sentiment analysis to news articles
news_df['sentiment_textblob'] = news_df['text'].apply(get_textblob_sentiment)

# Save sentiment-labeled datasets
tweets_df.to_csv("sentiment_tsla_tweets.csv", index=False)
news_df.to_csv("sentiment_tsla_news.csv", index=False)

print("Sentiment analysis complete! Results saved.")

Sentiment analysis complete! Results saved.


# SPARE CODE (sentiment snalysis)

In [36]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [37]:
# Load the datasets
tweets_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/cleaned_msft_tweets.csv")  # Processed tweets dataset
news_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/cleaned_msft_articles.csv")  # Processed news dataset

#print(tweets_df.columns)
#print(news_df.columns)

# Ensure text column is string type
tweets_df['Tweet'] = tweets_df['Tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

In [38]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [39]:
# Function to get sentiment using VADER for tweets
def get_sentiment_vader(text):
    score = sia.polarity_scores(text)['compound']
    return 1 if score > 0 else (-1 if score < 0 else 0)

In [40]:
# Function to get sentiment using TextBlob for news
def get_sentiment_textblob(text):
    score = TextBlob(text).sentiment.polarity
    return 1 if score > 0 else (-1 if score < 0 else 0)

In [41]:
tweets_df['auto_label'] = tweets_df['Tweet'].apply(get_sentiment_vader)
news_df['auto_label'] = news_df['text'].apply(get_sentiment_textblob)

In [42]:
# Merge both datasets for training
labeled_data = pd.concat([tweets_df[['Tweet', 'auto_label']], news_df[['text', 'auto_label']]])

In [43]:
# Convert sentiment labels to a non-negative integer format
label_mapping = {-1: 0, 0: 1, 1: 2}
labeled_data['auto_label'] = labeled_data['auto_label'].map(label_mapping)

# Convert labels to integer type
y = labeled_data['auto_label'].astype(int)

In [54]:
#all_text = pd.concat([tweets_df['Tweet'], news_df['text']], axis=0).dropna()

# Fit vectorizer on all available text
#vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
#vectorizer.fit(all_text)

vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)

# Ensure labeled_data exists and contains text
if 'text' not in labeled_data.columns:
    raise ValueError("Column 'text' not found in labeled_data. Check column names:", labeled_data.columns)

# Convert text data into numerical features
#X = vectorizer.fit_transform(labeled_data['text'])

# Ensure y is defined correctly
#y = labeled_data['auto_label'].astype(int)


all_text = pd.concat([labeled_data['text'], tweets_df['Tweet'], news_df['text']], axis=0).dropna()
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
vectorizer.fit(all_text)
X = vectorizer.fit_transform(labeled_data['text'])
y = labeled_data['auto_label'].astype(int)


In [47]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# Train Naïve Bayes Model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [49]:
# Train SVM Model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [50]:
# Evaluate Models
y_pred_nb = nb_model.predict(X_test)
y_pred_svm = svm_model.predict(X_test)

In [51]:
print("Naïve Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))

Naïve Bayes Accuracy: 0.4444444444444444
SVM Accuracy: 0.4603174603174603


In [53]:
print("Tweets dataset shape:", tweets_df.shape)
print("News dataset shape:", news_df.shape)
print("First few tweets:", tweets_df['Tweet'].head())
print("First few news articles:", news_df['text'].head())

# Check for missing values
print("Missing values in tweets:", tweets_df['Tweet'].isnull().sum())
print("Missing values in news:", news_df['text'].isnull().sum())


Tweets dataset shape: (0, 12)
News dataset shape: (313, 4)
First few tweets: Series([], Name: Tweet, dtype: object)
First few news articles: 0    voiceactivated helpers can automate lifes litt...
1    the sp  nasdaq and russell  are all in record ...
2    as china closes off cities affected by the gro...
3    the internet search giant became the fourth te...
4    the homerental company plans to consider the n...
Name: text, dtype: object
Missing values in tweets: 0
Missing values in news: 0


In [56]:
# Now transform new data
#tweets_df['sentiment_ml'] = nb_model.predict(vectorizer.transform(tweets_df['Tweet']))
#news_df['sentiment_ml'] = nb_model.predict(vectorizer.transform(news_df['text']))

#tweets_df['sentiment_ml'] = nb_model.predict(vectorizer.transform(tweets_df['Tweet']))
news_df['sentiment_ml'] = nb_model.predict(vectorizer.transform(news_df['text']))


ValueError: Found array with 0 sample(s) (shape=(0, 2417)) while a minimum of 1 is required by TfidfTransformer.

In [105]:
# Save final datasets with predicted sentiment
#tweets_df.to_csv("sentiment_msft_tweets.csv", index=False)
#news_df.to_csv("sentiment_tsla_news.csv", index=False)